In [1]:
import os
import sys

sys.path.append("../python")

import matplotlib.pyplot as plt
import numpy as np

In [2]:
from data import Dataloader
from models import RandomExampleModel
import metrics

In [3]:
# Loading the data
data_loader = Dataloader("../data/dataset_4") # Specify path to the dataset you want to load
X, Y = data_loader.load()


In [4]:
X_re = X
Y_re = Y

In [7]:
from gurobipy import *
import math
# Instanciation du modèle
m = Model("IntuitionV0")

PRECISION = 0.001
#Nombre de features
N_CRITERIA = len(X_re[0])

#Nombre de segment/breakpoints dans toutes les fonctions de score, possible de l'adapter pour chaque feature 
L = 5
#Nombre d'occurence dans le dataset/produits traités 
N_COMPARAISON = len(X_re)


#Nombre de clusters 
K=2

#Majorant contrainte 
M=4

#à définir clairement : 


criteria = [[[(m.addVar(name=f"u_{j}{l}{k}")) for k in range(K)] for l in range(L+1)]for j in range(N_CRITERIA) ]

sigma_Xplus = [[(m.addVar(name=f"sigma+{i}{k}")) for k in range(K) ] for i in range(N_COMPARAISON)]
sigma_Xminus = [[(m.addVar(name=f"sigma-{i}{k}")) for k in range(K)] for i in range(N_COMPARAISON)]
sigma_Yplus = [[(m.addVar(name=f"sigma+{i}{k}")) for k in range(K)] for i in range(N_COMPARAISON)]
sigma_Yminus = [[(m.addVar(name=f"sigma-{i}{k}")) for k in range(K)] for i in range(N_COMPARAISON)]

#sigma_Xplus = [[(m.addVar(name=f"sigma+{i}{k}")) for i in range(N_COMPARAISON)] for k in range(K)]
#sigma_Xminus = [[(m.addVar(name=f"sigma-{i}{k}")) for i in range(N_COMPARAISON)] for k in range(K)]
#sigma_Yplus = [[(m.addVar(name=f"sigma+{i}{k}")) for i in range(N_COMPARAISON)] for k in range(K)]
#sigma_Yminus = [[(m.addVar(name=f"sigma-{i}{k}")) for i in range(N_COMPARAISON)] for k in range(K)]


#sigma_Xplus = [(m.addVar(name=f"sigma+{i}{k}") for i in range(N_COMPARAISON))for k in range(K)]
#sigma_Xminus = [(m.addVar(name=f"sigma-{i}{k}") for i in range(N_COMPARAISON))for k in range(K)]

#sigma_Yplus = [(m.addVar(name=f"sigma+{i}{k}") for i in range(N_COMPARAISON))for k in range(K)]
#sigma_Yminus = [(m.addVar(name=f"sigma-{i}{k}") for i in range(N_COMPARAISON))for k in range(K)]

z_binary = [[m.addVar(name=f"z_{i}_{k}") for k in range(K) ] for i in range(N_COMPARAISON)]

#hyperparamètre pour le modèle
epsilon = PRECISION
# maj du modèle
m.update()

#print(r"120", criteria[1][2][0])
#print(r"230", criteria[2][3][0])
#print(r"140", criteria[1][4][0])



#récupère les minimum et maximum de chaque feature 
#mins = (min(X_re.min(axis=0), Y_re.min(axis=0))).any()
#maxs = (max(X_re.max(axis=0), Y_re.max(axis=0))).any()

min_int_X = X_re.min(axis=0)
min_int_Y = Y_re.min(axis=0)
mins = np.min(np.vstack((min_int_X, min_int_Y)), axis = 0)

max_int_X = X_re.max(axis=0)
max_int_Y = Y_re.max(axis=0)
maxs = np.max(np.vstack((max_int_X, max_int_Y)), axis = 0)

def segment(i, j, X):
    x = X[i][j]
    #floor permet de faire l'arrondi 
    #donne un chiffre entre 0 et L (ici 5) qui donne l'appartenance à l'un des 6 segment possible 
    #X[i] correspond à une occurence d'un produit traité 
    #j correspond à l'indice de feature 
    #correspond au commentaire de la question 2
    return math.floor(L * (x - mins[j]) / (maxs[j] - mins[j]))

#cette fonction précédente permettra a priori de trouver le l de la fonction suivante

#l correspond au tronçon sur lequel est la valeur 
#retourne l'abscisse de gauche ? du tronçon l 
#appelé xik dans question 3
def x_seg(j, seg):
    return mins[j] + seg * (maxs[j] - mins[j]) / L

#calcule la valeur si(xij)
#cad le score partiel pour un feature donné 
# ?? on obtient le score à partir d'une forme ressemblant au développement limité autour de la valeur de la feature sur le tronçon

def u(i,j,k,X,eval : bool = False):
    get_val = (lambda v: v.X) if eval else (lambda v: v)
    x = X[i][j]
    #segm pour le numéro de tronçon
    
    segm = segment(i, j, X)
    #print(r"segm",segm)
    x_segm = x_seg(j, segm)
    #print(r"x_segm", x_segm)
    x_segm_1 = x_seg(j, segm +1)
    #print(r"x_segm_1", x_segm_1)


    if x == maxs[j]:
        return get_val(criteria[j][-1][k]) 
    
    S=0
    S = get_val(criteria[j][segm][k])
    slope = (x - x_segm)/(x_segm_1 - x_segm)
    width = get_val(criteria[j][segm+1][k]) - get_val(criteria[j][segm][k])
    S =+ slope*width

    return S


#définissons la somme des fonctions de score pour un produit (de rang i)
def s(i, X, k, eval : bool = False):
    if not eval : 
        return quicksum(u(i,j,k,X,False) for j in range(N_CRITERIA))
    else : 
        return sum(u(i,j,k,X,False) for j in range(N_CRITERIA))



# Ajout des contraintes

#Documentation Gurobi propre



# contraintes de préférence des universités
for k in range(K): 

    for i in range(N_COMPARAISON) :
        #print(r"boucle contrainte",i)
        A = s(i, X_re, k, False) 
        C = sigma_Xplus[i][k] 
        E = sigma_Xminus[i][k] 
        D = s(i, Y_re, k, False) - sigma_Yplus[i][k] + sigma_Yminus[i][k] + epsilon
        B = (M*z_binary[i][k]+ epsilon)
        

        m.addConstr((s(i, X_re, k, False) - sigma_Xplus[i][k] + sigma_Xminus[i][k]) - (s(i, Y_re, k, False) - sigma_Yplus[i][k] + sigma_Yminus[i][k] + epsilon)<= (M*z_binary[i][k]+ epsilon))
        m.addConstr(M*(1-z_binary[i][k])<= (s(i, X_re, k, False) - sigma_Xplus[i][k] + sigma_Xminus[i][k]) - (s(i, Y_re, k, False) - sigma_Yplus[i][k] + sigma_Yminus[i][k] + epsilon))

        #m.addConstr(M*(1-z_binary[i][k])<= (s(X_re[i], k) - sigma_Xplus[i][k] + sigma_Xminus[i][k]) - (s(Y_re[i], k) - sigma_Yplus[i][k] + sigma_Yminus[i][k] + epsilon))
        #m.addConstr((s(X_re[i], k) - sigma_Xplus[i][k] + sigma_Xminus[i][k]) - (s(Y_re[i], k) - sigma_Yplus[i][k] + sigma_Yminus[i][k] + epsilon)<= (M*z_binary[i][k]+ epsilon))
        #m.addConstr(s(X_re[i]) - sigma_Xplus[i] + sigma_Xminus[i] >=  s(Y_re[i]) - sigma_Yplus[i] + sigma_Yminus[i] + epsilon)


# Constraint for sigma_Xminus
for k in range(K):
    for i in range(N_COMPARAISON):
        m.addConstr(sigma_Xminus[i][k] >= 0)

# Constraint for sigma_Xplus
for k in range(K):
    for i in range(N_COMPARAISON):
        m.addConstr(sigma_Xplus[i][k] >= 0)

# Constraint for sigma_Yminus
for k in range(K):
    for i in range(N_COMPARAISON):
        m.addConstr(sigma_Yminus[i][k] >= 0)

# Constraint for sigma_Yplus
for k in range(K):
    for i in range(N_COMPARAISON):
        m.addConstr(sigma_Yplus[i][k] >= 0)

#for sig in sigma_Xminus + sigma_Xplus + sigma_Yminus + sigma_Yplus:
#    m.addConstr(sig >= 0)

# contrainte 3
for k in range(K) :
    for l in range(L):
        for j in range(N_CRITERIA):
            m.addConstr(criteria[j][l+1][k] - criteria[j][l][k] >= epsilon)


# contrainte 4
for k in range(K):
    for j in range(N_CRITERIA):
        m.addConstr(criteria[j][L][k] ==1)
    
#m.addConstr(s(i, X, k, False) == 1)

# contrainte 5
for k in range(K):
    for j in range(N_CRITERIA):
        #ajustement arbitraire avec 2222 car u() traite toujours des matrices
        mins_ada = np.vstack([np.array([ 2, 2, 2, 2]), mins])
        
        print(mins_ada)
        #mins correspond aux plus petites valeurs possibles pour chaque critère
        m.addConstr(u(1, j,k, mins_ada, False) == 0)


        
# Flatten the lists of lists into a single list
sigma_Xplus_flat = [var for sublist in sigma_Xplus for var in sublist]
sigma_Xminus_flat = [var for sublist in sigma_Xminus for var in sublist]
sigma_Yminus_flat = [var for sublist in sigma_Yminus for var in sublist]
sigma_Yplus_flat = [var for sublist in sigma_Yplus for var in sublist]

# Objective function
m.setObjective(sum(sigma_Xplus_flat) + sum(sigma_Xminus_flat) + sum(sigma_Yminus_flat) + sum(sigma_Yplus_flat), GRB.MINIMIZE)


# Fonction Objectif
#m.setObjective(sum(sigma_Xplus) + sum(sigma_Xminus) + sum(sigma_Yminus) + sum(sigma_Yplus), GRB.MINIMIZE)         
# Paramétrage (mode mute)
m.Params.Presolve = 0
# Résolution du PL
m.optimize()
#print("La solution optimale est (v0, v1) = {} avec pour objectif z = {}".format((v0.x, v1.x), m.objVal))
print(criteria)

[[2.00000000e+00 2.00000000e+00 2.00000000e+00 2.00000000e+00]
 [3.82003667e-04 2.30327233e-05 1.63227181e-03 3.51066072e-04]]
[[2.00000000e+00 2.00000000e+00 2.00000000e+00 2.00000000e+00]
 [3.82003667e-04 2.30327233e-05 1.63227181e-03 3.51066072e-04]]
[[2.00000000e+00 2.00000000e+00 2.00000000e+00 2.00000000e+00]
 [3.82003667e-04 2.30327233e-05 1.63227181e-03 3.51066072e-04]]
[[2.00000000e+00 2.00000000e+00 2.00000000e+00 2.00000000e+00]
 [3.82003667e-04 2.30327233e-05 1.63227181e-03 3.51066072e-04]]
[[2.00000000e+00 2.00000000e+00 2.00000000e+00 2.00000000e+00]
 [3.82003667e-04 2.30327233e-05 1.63227181e-03 3.51066072e-04]]
[[2.00000000e+00 2.00000000e+00 2.00000000e+00 2.00000000e+00]
 [3.82003667e-04 2.30327233e-05 1.63227181e-03 3.51066072e-04]]
[[2.00000000e+00 2.00000000e+00 2.00000000e+00 2.00000000e+00]
 [3.82003667e-04 2.30327233e-05 1.63227181e-03 3.51066072e-04]]
[[2.00000000e+00 2.00000000e+00 2.00000000e+00 2.00000000e+00]
 [3.82003667e-04 2.30327233e-05 1.63227181e-03 3